In [33]:
import pandas as pd
from sqlalchemy import create_engine
from dotenv import load_dotenv
import os

In [34]:
load_dotenv("../credentials.env") #Here goes the credentials for your Database

db_username = os.getenv("DB_USERNAME")
db_password = os.getenv("DB_PASSWORD")
db_host = os.getenv("DB_HOST")
db_port = os.getenv("DB_PORT")
db_name = os.getenv("DB_NAME")

engine = create_engine(f"postgresql://{db_username}:{db_password}@{db_host}:{db_port}/{db_name}")

query = "SELECT * FROM clean_awards"

awards = pd.read_sql_query(query, engine)
awards.head()

,year,title,published_at,updated_at,category,track_name,artists,workers,winner
0,2019,62nd Annual GRAMMY Awards (2019),2020-05-19 12:10:28+00:00,2020-05-19 12:10:28+00:00,Record Of The Year,bad guy,billie eilish,"Finneas O'Connell, producer; Rob Kinelski & Fi...",True
1,2019,62nd Annual GRAMMY Awards (2019),2020-05-19 12:10:28+00:00,2020-05-19 12:10:28+00:00,Record Of The Year,"hey, ma",bon iver,"BJ Burton, Brad Cook, Chris Messina & Justin V...",True
2,2019,62nd Annual GRAMMY Awards (2019),2020-05-19 12:10:28+00:00,2020-05-19 12:10:28+00:00,Record Of The Year,7 rings,ariana grande,"Charles Anderson, Tommy Brown, Michael Foster ...",True
3,2019,62nd Annual GRAMMY Awards (2019),2020-05-19 12:10:28+00:00,2020-05-19 12:10:28+00:00,Record Of The Year,hard place,h.e.r.,"Rodney “Darkchild” Jerkins, producer; Joseph H...",True
4,2019,62nd Annual GRAMMY Awards (2019),2020-05-19 12:10:28+00:00,2020-05-19 12:10:28+00:00,Record Of The Year,talk,khalid,"Disclosure & Denis Kosiak, producers; Ingmar C...",True


In [35]:
spotify = pd.read_csv("../data/spotify_dataset_clean.csv", delimiter=',')
spotify.head()

,Unnamed: 0,track_id,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre
0,0,5SuOikwiRyPMVoIQDJUgSV,gen hoshino,Comedy,comedy,73,230666,False,0.676,0.4610,...,-6.746,0,0.1430,0.0322,0.000001,0.3580,0.715,87.917,4,acoustic
1,1,4qPNDBW1i3p13qLCt0Ki3A,ben woodward,Ghost (Acoustic),ghost - acoustic,55,149610,False,0.420,0.1660,...,-17.235,1,0.0763,0.9240,0.000006,0.1010,0.267,77.489,4,acoustic
2,2,1iJBSr7s7jYXzM8EGcbK5b,ingrid michaelson;zayn,To Begin Again,to begin again,57,210826,False,0.438,0.3590,...,-9.734,1,0.0557,0.2100,0.000000,0.1170,0.120,76.332,4,acoustic
3,3,6lfxq3CG4xtTiEg7opyCyx,kina grannis,Crazy Rich Asians (Original Motion Picture Sou...,can't help falling in love,71,201933,False,0.266,0.0596,...,-18.515,1,0.0363,0.9050,0.000071,0.1320,0.143,181.740,3,acoustic
4,4,5vjLSffimiIP26QG5WcN2K,chord overstreet,Hold On,hold on,82,198853,False,0.618,0.4430,...,-9.681,1,0.0526,0.4690,0.000000,0.0829,0.167,119.949,4,acoustic


In [36]:
# Realizar el merge
merged_df = pd.merge(spotify, awards, on=['artists', 'track_name'], how='left')

In [37]:
merged_df.head(4)

,Unnamed: 0,track_id,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,...,tempo,time_signature,track_genre,year,title,published_at,updated_at,category,workers,winner
0,0,5SuOikwiRyPMVoIQDJUgSV,gen hoshino,Comedy,comedy,73,230666,False,0.676,0.4610,...,87.917,4,acoustic,NaN,NaN,NaT,NaT,NaN,NaN,NaN
1,1,4qPNDBW1i3p13qLCt0Ki3A,ben woodward,Ghost (Acoustic),ghost - acoustic,55,149610,False,0.420,0.1660,...,77.489,4,acoustic,NaN,NaN,NaT,NaT,NaN,NaN,NaN
2,2,1iJBSr7s7jYXzM8EGcbK5b,ingrid michaelson;zayn,To Begin Again,to begin again,57,210826,False,0.438,0.3590,...,76.332,4,acoustic,NaN,NaN,NaT,NaT,NaN,NaN,NaN
3,3,6lfxq3CG4xtTiEg7opyCyx,kina grannis,Crazy Rich Asians (Original Motion Picture Sou...,can't help falling in love,71,201933,False,0.266,0.0596,...,181.740,3,acoustic,NaN,NaN,NaT,NaT,NaN,NaN,NaN


## Transformations

In [38]:
#Object to string
merged_df = merged_df.astype({col: 'string' for col in df.select_dtypes(include='object').columns}) #Object to string
merged_df['winner'] = merged_df['winner'].fillna(False)

C:\Users\Usuario\AppData\Local\Temp\ipykernel_1852\1143245274.py:3: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  merged_df['winner'] = merged_df['winner'].fillna(False)


## Only the winners

In [39]:
filtered_rows = merged_df[merged_df['winner'].notna()]

# Mostrar las filas filtradas
filtered_rows.head(3)

,Unnamed: 0,track_id,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,...,tempo,time_signature,track_genre,year,title,published_at,updated_at,category,workers,winner
0,0,5SuOikwiRyPMVoIQDJUgSV,gen hoshino,Comedy,comedy,73,230666,False,0.676,0.461,...,87.917,4,acoustic,NaN,NaN,NaT,NaT,NaN,NaN,False
1,1,4qPNDBW1i3p13qLCt0Ki3A,ben woodward,Ghost (Acoustic),ghost - acoustic,55,149610,False,0.420,0.166,...,77.489,4,acoustic,NaN,NaN,NaT,NaT,NaN,NaN,False
2,2,1iJBSr7s7jYXzM8EGcbK5b,ingrid michaelson;zayn,To Begin Again,to begin again,57,210826,False,0.438,0.359,...,76.332,4,acoustic,NaN,NaN,NaT,NaT,NaN,NaN,False


In [40]:
filtered_rows.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 114062 entries, 0 to 114061
Data columns (total 28 columns):
 #   Column            Non-Null Count   Dtype              
---  ------            --------------   -----              
 0   Unnamed: 0        114062 non-null  int64              
 1   track_id          114062 non-null  string             
 2   artists           114062 non-null  string             
 3   album_name        114062 non-null  string             
 4   track_name        114062 non-null  string             
 5   popularity        114062 non-null  int64              
 6   duration_ms       114062 non-null  int64              
 7   explicit          114062 non-null  bool               
 8   danceability      114062 non-null  float64            
 9   energy            114062 non-null  float64            
 10  key               114062 non-null  int64              
 11  loudness          114062 non-null  float64            
 12  mode              114062 non-null  int64    

## Save

In [41]:
merged_df.to_csv("../data/transformed_dataset.csv", index=False)